In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from joblib import dump, load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [16]:
data = pd.read_csv("training_data_with_player_direction.csv")

/var/folders/32/p6kbh9zj1w96fk3ml32ftvph0000gn/T/ipykernel_61256/764269968.py:1: DtypeWarning: Columns (389) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("training_data_with_player_direction.csv")


In [23]:
# Transform Positions
data_transformed = data
data_transformed['play_direction_numerical'] = 1
data_transformed.loc[data_transformed['playDirection'] == 'left', 'play_direction_numerical'] = -1
delta_cols = []
for var_type in ['x', 'y', 's', 'a', 'dir', 'dis', 'o']: # note that magnitude vars (like speed and acceleration) are omitted.
    for pos in ['LT', 'LG', 'C', 'RG', 'RT']:
        col_name = var_type + "_0_" + pos
        for time in range(1, 11): # exclude 0, include 1.0
            time = round(time * 0.1, 1) # working with decimals
            time_str = str(time)
            if time == 1:
                time_str = '1'
            col_name = var_type + "_" + str(time_str) + "_" + pos
            print(col_name)
            prev_time = round(time - 0.1, 1)
            prev_time_str = str(prev_time)
            if prev_time == 0:
                prev_time_str = '0' # not '0.0'
            prev_time_col_name = var_type + "_" + str(prev_time_str) + "_" + pos
            delta_col_name = 'delta_' + col_name
            delta_cols.append(delta_col_name)
            data_transformed[delta_col_name] = (data_transformed[col_name] - data_transformed[prev_time_col_name]) * data_transformed['play_direction_numerical']

# data_transformed.to_csv("data_transformed.csv")
# data_transformed[['x_0.1_LT', 'x_0_LT', 'delta_x_0.1_LT', 'playDirection']] # quick check

x_0.1_LT
x_0.2_LT
x_0.3_LT
x_0.4_LT
x_0.5_LT
x_0.6_LT
x_0.7_LT
x_0.8_LT
x_0.9_LT
x_1_LT
x_0.1_LG
x_0.2_LG
x_0.3_LG
x_0.4_LG
x_0.5_LG
x_0.6_LG
x_0.7_LG
x_0.8_LG
x_0.9_LG
x_1_LG
x_0.1_C
x_0.2_C
x_0.3_C
x_0.4_C
x_0.5_C
x_0.6_C
x_0.7_C
x_0.8_C
x_0.9_C
x_1_C
x_0.1_RG
x_0.2_RG
x_0.3_RG
x_0.4_RG
x_0.5_RG
x_0.6_RG
x_0.7_RG
x_0.8_RG
x_0.9_RG
x_1_RG
x_0.1_RT
x_0.2_RT
x_0.3_RT
x_0.4_RT
x_0.5_RT
x_0.6_RT
x_0.7_RT
x_0.8_RT
x_0.9_RT
x_1_RT
y_0.1_LT
y_0.2_LT
y_0.3_LT
y_0.4_LT
y_0.5_LT
y_0.6_LT
y_0.7_LT
y_0.8_LT
y_0.9_LT
y_1_LT
y_0.1_LG
y_0.2_LG
y_0.3_LG
y_0.4_LG
y_0.5_LG
y_0.6_LG
y_0.7_LG
y_0.8_LG
y_0.9_LG
y_1_LG
y_0.1_C
y_0.2_C
y_0.3_C
y_0.4_C
y_0.5_C
y_0.6_C
y_0.7_C
y_0.8_C
y_0.9_C
y_1_C
y_0.1_RG
y_0.2_RG
y_0.3_RG
y_0.4_RG
y_0.5_RG
y_0.6_RG
y_0.7_RG
y_0.8_RG
y_0.9_RG
y_1_RG
y_0.1_RT
y_0.2_RT
y_0.3_RT
y_0.4_RT
y_0.5_RT
y_0.6_RT
y_0.7_RT
y_0.8_RT
y_0.9_RT
y_1_RT
s_0.1_LT
s_0.2_LT
s_0.3_LT
s_0.4_LT
s_0.5_LT
s_0.6_LT
s_0.7_LT
s_0.8_LT
s_0.9_LT
s_1_LT
s_0.1_LG
s_0.2_LG
s_0.3_LG
s_0.4_LG
s_0.5_LG
s_0.6_L

In [7]:
# Principal Component Analysis
cols_of_interest = ['action_time'] + delta_cols
data_all = data_transformed.dropna()
data = data_all.select_dtypes(['number'])
data = data[cols_of_interest]
x = data.loc[:, data.columns != 'action_time']
y = data['action_time']
pca = PCA(n_components = 35) # in theory, one per variable per position
print("pca fitting")
principal_components = pca.fit_transform(x)
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
principal_components_df.to_csv("35_principal_components_time_to_throw.csv")

pca fitting
[[ 2.47663996e+00  2.30439355e+00  1.39336490e+00 ...  1.83541398e+00
   1.15609001e+01  6.40240681e+00]
 [ 3.49963450e+02  4.39020795e+01 -3.49058678e+01 ... -9.86171649e+00
  -1.47761719e+01 -5.69751116e+01]
 [ 5.82947248e+00 -1.79136972e+00  2.68771930e+00 ... -5.05975311e-01
   1.96779847e-01  2.14032592e+00]
 ...
 [-6.83527646e+00  1.41023890e+00 -1.11108502e+01 ... -4.19967188e+00
  -2.62145140e+00  5.41634109e+00]
 [ 7.74738969e+00  7.62583576e-01 -1.19107512e+01 ...  5.44683427e+00
  -2.67278629e+00  4.84294723e+00]
 [-1.41123399e+01 -2.32415477e+01 -8.44827046e+00 ...  3.63294418e+00
   3.73754093e-01  8.46291489e+00]]


In [8]:
# Prep Train and Test Data
label = data['action_time']
principal_components_df['action_time'] = label
X = data.loc[:, data.columns != 'action_time']
y = data['action_time']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print("completed train-test split")

completed train-test split


In [9]:
print(datetime.now())
random_grid = { # FLAG
    'bootstrap': [True, False],
    'max_depth': [5, 10, 20, 50],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [3, 5, 10],
    'min_samples_split': [3, 5, 15],
    'n_estimators': [250, 500, 1000, 2000],
}

rf_tuning = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf_tuning, param_distributions = random_grid, n_iter = 30, cv = 3, verbose = 2, random_state = 42, n_jobs = -1)
rf_fit_output = rf_random.fit(x_train, y_train)
print(datetime.now())

2022-12-09 00:44:42.085947
Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] END bootstrap=False, max_depth=5, max_features=log2, min_samples_leaf=5, min_samples_split=15, n_estimators=1000; total time=   1.6s
[CV] END bootstrap=False, max_depth=5, max_features=log2, min_samples_leaf=5, min_samples_split=15, n_estimators=1000; total time=   1.8s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=3, min_samples_split=15, n_estimators=1000; total time=   2.2s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=3, min_samples_split=15, n_estimators=1000; total time=   2.4s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=3, min_samples_split=15, n_estimators=1000; total time=   2.5s
[CV] END bootstrap=False, max_depth=5, max_features=log2, min_samples_leaf=5, min_samples_split=15, n_estimators=1000; total time=   2.3s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=5, min_sa

In [10]:
rf_tuned = rf_fit_output.best_estimator_
rf_tuned.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=10, max_features='log2',
                      min_samples_leaf=10, min_samples_split=5,
                      n_estimators=500)

In [11]:
# Use the forest's predict method on the test data
predictions = rf_tuned.predict(x_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
print('MSE: ', mean_squared_error(y_test, predictions))
print('R^2: ', rf_tuned.score(x_test, y_test))

Mean Absolute Error: 0.82 degrees.
Accuracy: 71.54 %.
MSE:  1.7655242782531237
R^2:  0.056831211165461304


In [13]:
# Import tools needed for visualization
# from sklearn.tree import export_graphviz
# import pydot# Pull out one tree from the forest
# tree = rf_tuned.estimators_[5]# Import tools needed for visualization
# from sklearn.tree import export_graphviz
# import pydot# Pull out one tree from the forest
# tree = rf_tuned.estimators_[5]# Export the image to a dot file
# features = [col for col in data.columns if col != 'action_time']
# export_graphviz(tree, out_file = 'tree.dot', feature_names = features, rounded = True, precision = 1)# Use dot file to create a graph
# (graph, ) = pydot.graph_from_dot_file('tree.dot')# Write graph to a png file
# # print(graph)
# graph.write_png('tree.png')

FileNotFoundError: [Errno 2] "dot" not found in path.

In [ ]:
test_set_with_predictions = pd.DataFrame(columns = {'actual': y_test, 'predicted': predictions})
test_set_with_predictions['actual'] = y_test
test_set_with_predictions['predicted'] = predictions
test_set_with_predictions.to_csv("test_set_with_predictions.csv")
#for i in range(len(y_test)):
#    print(y_test[i], predictions[i])